<a href="https://colab.research.google.com/github/MinKhantKyaw5006/LLM-from-Scratch/blob/main/Practice1/Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Creating Tokens

In [ ]:
print("Hello")

Hello
